In [11]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import tqdm
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder 

cnx = sqlite3.connect('F:\py_assignment\project1\database.sqlite')
df = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)

input_data=df.copy(deep=True)

#Number of Nan's for each bject
input_data.isnull().sum(axis=1).sort_values(ascending=False)[:15]

input_data.isnull().sum(axis=0).sort_values(ascending=False)

#dropna=False makes nunique treat NaNs  as a distinct value
feats_counts= input_data.nunique(dropna= False)

feats_counts.sort_values()[:10]

constant_features= feats_counts.loc[feats_counts==1].index.tolist()

print(constant_features)

input_data.drop(constant_features,axis=1,inplace=True)

#remove duplicated features
input_data.fillna('NaN',inplace=True)

input_data_enc=pd.DataFrame(index=input_data.index)

for col in tqdm_notebook(input_data.columns):
    input_data_enc[col]=input_data[col].factorize()[0]

#duplicate columns
dupes=input_data.duplicated()
#No duplicates fund
print(sum(dupes))

#converting date to julian date
input_data['newdate']=pd.to_datetime(input_data['date'])
input_data['newdate']=pd.to_datetime(0,unit='s').to_julian_date()

input_data=input_data.drop(['date'],axis=1)

nunique= input_data.nunique(dropna=False)
nunique

mask=(nunique.astype(float)/input_data.shape[0]<0.8) & (nunique.astype(float)/input_data.shape[0]<0.4)
input_data.loc[:22,mask]

dataset_target= input_data['overall_rating']
input_data.drop('overall_rating',axis=1, inplace=True)

cat_cols=list(input_data.select_dtypes(include=['object']).columns)
num_cols=list(input_data.select_dtypes(exclude=['object']).columns)

num_cols

N=StandardScaler()
N.fit(input_data[num_cols])
input_data_norm=N.transform(input_data[num_cols])

input_data[cat_cols].head()

input_data_lenc=pd.DataFrame({})
le=LabelEncoder()
for c in cat_cols:
        input_data_lenc[c] = le.fit_transform(list(input_data[c]))
        
input_data_lenc.head()
input_data_norm[:1]
dataset_features= np.concatenate((input_data_norm , input_data_lenc),axis=1)

kfold= StratifiedKFold(n_splits=10)

dataset_target= dataset_target.astype('str')

DTC=DecisionTreeClassifier(max_depth=3)
cv_results= cross_val_score(DTC,dataset_features,dataset_target.values,cv=kfold,scoring="accuracy")

#prints accuracy of 0.12
cv_results.mean()


[]


A Jupyter Widget


0


C:\Users\naveen.kudari\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.12901058543599792